In [1]:
import json
import csv
import requests
import locale
from datetime import datetime

In [2]:
# Configurando locale para formato brasileiro
try:
    locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
except locale.Error:
    try:
        locale.setlocale(locale.LC_ALL, 'Portuguese_Brazil.1252')
    except locale.Error:
        print("Aviso: Não foi possível definir locale brasileiro. Usando formato padrão.")


In [3]:

def formatar_valor_brasileiro(valor):
    """Formata valores numéricos para o padrão brasileiro"""
    if valor is None or valor == 0:
        return "0,00"
    
    try:
        # Se o valor for uma string, tenta converter para float
        if isinstance(valor, str):
            valor = float(valor.replace(',', '.'))
        
        # Formata com 2 casas decimais e separador brasileiro
        return f"{valor:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')
    except (ValueError, TypeError):
        return str(valor)

def formatar_taxa_brasileira(taxa):
    """Formata taxas para o padrão brasileiro (percentual)"""
    if taxa is None or taxa == 0:
        return "0,00%"
    
    try:
        if isinstance(taxa, str):
            taxa = float(taxa.replace(',', '.'))
        
        # Formata como percentual com 2 casas decimais
        return f"{taxa:.2f}%".replace('.', ',')
    except (ValueError, TypeError):
        return str(taxa)
def buscar_titulo(dados_tesouro, nome_titulo):
    """Busca um título específico na lista de títulos do Tesouro Direto"""
    for titulo in dados_tesouro['TrsrBdTradgList']:
        if titulo['TrsrBd']['nm'].lower() == nome_titulo.lower():
            return titulo
    return None


In [4]:
def buscar_titulo(dados_tesouro, nome_titulo):
    """Busca um título específico na lista de títulos do Tesouro Direto"""
    for titulo in dados_tesouro['TrsrBdTradgList']:
        if titulo['TrsrBd']['nm'].lower() == nome_titulo.lower():
            return titulo
    return None

def extrair_informacoes_titulo(titulo):
    """Extrai todas as informações importantes de um título"""
    bond = titulo['TrsrBd']
    bond_type = titulo['TrsrBdType']
    
    # Formatando a data de vencimento
    data_vencimento = datetime.fromisoformat(bond['mtrtyDt'].replace('T00:00:00', ''))
    
    info = {
        'nome': bond['nm'],
        'codigo': bond['cd'],
        'isin_code': bond['isinCd'],
        'data_vencimento': data_vencimento.strftime('%d/%m/%Y'),
        'tipo': bond_type['nm'],
        'indice_financeiro': bond['FinIndxs']['nm'],
        'valor_minimo_investimento': formatar_valor_brasileiro(bond['minInvstmtAmt']),
        'valor_unitario_investimento': formatar_valor_brasileiro(bond['untrInvstmtVal']),
        'taxa_anual_investimento': formatar_taxa_brasileira(bond['anulInvstmtRate']),
        'valor_minimo_resgate': formatar_valor_brasileiro(bond['minRedVal']),
        'valor_unitario_resgate': formatar_valor_brasileiro(bond['untrRedVal']),
        'taxa_anual_resgate': formatar_taxa_brasileira(bond['anulRedRate']),
        'quantidade_minima_resgate': formatar_valor_brasileiro(bond['minRedQty']),
        'paga_juros_semestrais': 'Sim' if bond['semiAnulIntrstInd'] else 'Não',
        'caracteristicas': bond['featrs'].strip().replace('\r\n', ' ').replace('\n', ' '),
        'estabilidade_investimento': bond.get('invstmtStbl', '').strip().replace('\r\n', ' ').replace('\n', ' ') if bond.get('invstmtStbl') else '',
        'renda_recebida': bond.get('rcvgIncm', '').strip().replace('\r\n', ' ').replace('\n', ' ') if bond.get('rcvgIncm') else '',
        'selic_code': titulo['SelicCode']
    }
    
    # Adicionando informações específicas para títulos Renda+ e Educa+
    if bond.get('BusSegmt'):
        info['segmento'] = bond['BusSegmt']['nm']
        info['quantidade_amortizacao'] = bond['amortQuotQty']
        if bond.get('convDt'):
            data_conversao = datetime.fromisoformat(bond['convDt'].replace('T00:00:00', ''))
            info['data_conversao'] = data_conversao.strftime('%d/%m/%Y')
        else:
            info['data_conversao'] = ''
    else:
        info['segmento'] = ''
        info['quantidade_amortizacao'] = ''
        info['data_conversao'] = ''
    
    return info

In [5]:

def carregar_dados_local(caminho_arquivo):
    """Carrega dados do arquivo JSON local"""
    try:
        with open(caminho_arquivo, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Arquivo {caminho_arquivo} não encontrado")
        return None
    except json.JSONDecodeError:
        print(f"Erro ao decodificar JSON do arquivo {caminho_arquivo}")
        return None

def carregar_dados_url(url):
    """Carrega dados da URL"""
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Erro ao buscar dados da URL {url}: {e}")
        return None
    except json.JSONDecodeError:
        print(f"Erro ao decodificar JSON da URL {url}")
        return None


In [6]:

# Lista dos títulos desejados
titulos_desejados = [
    'Tesouro IPCA+ 2035',
    'Tesouro IPCA+ 2045', 
    'Tesouro IPCA+ com Juros Semestrais 2032',
    'Tesouro IPCA+ com Juros Semestrais 2035',
    'Tesouro IPCA+ com Juros Semestrais 2040',
    'Tesouro IPCA+ com Juros Semestrais 2055',
    'Tesouro IPCA+ 2029'
]

# URLs e caminhos para os dados
url_github = "https://raw.githubusercontent.com/BDonadelli/TD/refs/heads/main/data/tesouro_direto.json"
url_oficial = "https://www.tesourodireto.com.br/json/br/com/b3/tesourodireto/service/api/treasurybondsinfo.json"
caminho_local = "/home/yair/GHub/TD/data/tesouro_direto.json"

# Tentativa de carregar os dados (prioridade: local -> GitHub -> oficial)
dados = carregar_dados_local(caminho_local)

if dados is None:
    print("Tentando carregar dados do GitHub...")
    dados = carregar_dados_url(url_github)

if dados is None:
    print("Tentando carregar dados da API oficial...")
    dados = carregar_dados_url(url_oficial)

if dados is None:
    print("Erro: Não foi possível carregar os dados de nenhuma fonte")
    # return

# Extraindo a resposta dos dados
if 'response' in dados:
    dados_tesouro = dados['response']
else:
    dados_tesouro = dados

# Lista para armazenar as informações extraídas
informacoes_titulos = []

# Processando cada título desejado
for nome_titulo in titulos_desejados:
    print(f"Processando: {nome_titulo}")
    titulo = buscar_titulo(dados_tesouro, nome_titulo)
    
    if titulo:
        info = extrair_informacoes_titulo(titulo)
        informacoes_titulos.append(info)
        print(f"✓ {nome_titulo} encontrado")
    else:
        print(f"✗ {nome_titulo} não encontrado")

# Salvando os dados em CSV
if informacoes_titulos:
    nome_arquivo_csv = f"tesouro_direto_dados.csv"
    # nome_arquivo_csv = f"tesouro_direto_dados_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"

    # Definindo os cabeçalhos do CSV
    cabecalhos = [
        'nome', 'codigo', 'isin_code', 'data_vencimento', 'tipo', 'indice_financeiro',
        'valor_minimo_investimento', 'valor_unitario_investimento', 'taxa_anual_investimento',
        'valor_minimo_resgate', 'valor_unitario_resgate', 'taxa_anual_resgate',
        'quantidade_minima_resgate', 'paga_juros_semestrais', 'caracteristicas',
        'estabilidade_investimento', 'renda_recebida', 'selic_code', 'segmento',
        'quantidade_amortizacao', 'data_conversao'
    ]
    
    with open("data/"+nome_arquivo_csv, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=cabecalhos, delimiter=';')
        writer.writeheader()
        writer.writerows(informacoes_titulos)
    
    print(f"\n✓ Dados salvos em: {nome_arquivo_csv}")
    print(f"Total de títulos processados: {len(informacoes_titulos)}")
    
    # Mostrando informações de status do mercado
    if 'TrsrBondMkt' in dados_tesouro:
        mercado = dados_tesouro['TrsrBondMkt']
        print(f"\nStatus do mercado: {mercado['sts']}")
        print(f"Última atualização: {mercado['qtnDtTm']}")
else:
    print("Nenhum título foi encontrado para processar")



Processando: Tesouro IPCA+ 2035
✓ Tesouro IPCA+ 2035 encontrado
Processando: Tesouro IPCA+ 2045
✓ Tesouro IPCA+ 2045 encontrado
Processando: Tesouro IPCA+ com Juros Semestrais 2032
✓ Tesouro IPCA+ com Juros Semestrais 2032 encontrado
Processando: Tesouro IPCA+ com Juros Semestrais 2035
✓ Tesouro IPCA+ com Juros Semestrais 2035 encontrado
Processando: Tesouro IPCA+ com Juros Semestrais 2040
✓ Tesouro IPCA+ com Juros Semestrais 2040 encontrado
Processando: Tesouro IPCA+ com Juros Semestrais 2055
✓ Tesouro IPCA+ com Juros Semestrais 2055 encontrado
Processando: Tesouro IPCA+ 2029
✓ Tesouro IPCA+ 2029 encontrado

✓ Dados salvos em: tesouro_direto_dados.csv
Total de títulos processados: 7

Status do mercado: Fechado
Última atualização: 2025-08-15T18:01:03.483
